Preprocessing :

*   1) Hair removal

*   2) Shade removal

*   3) Contour detection
*   4) Histogram equalizer
*   5) Débruitage
*   6) Détection de symétrie



**Hair Removal**

In [ ]:
import cv2

"""
Most images of skin lesions have unwanted elements, such as shadows and hairs, which can make it difficult to segment
the lesion and introduce erroneous information on its characteristics. Therefore, it is necessary to apply some
artificial vision techniques to eliminate any noise component.

Body hair is one of the factors that can affect lesion segmentation. For the detection and removal of hairs, a
pre-processing technique called DullRazor is used, which consists of applying a series of morphological operations
to the image in order to generate a mask that contains the hairs. The steps to apply the DullRazor algorithm are:

1) Convert the original image to grayscale.
2) Closing to the grayscale image, using a linear or cross-shaped kernel.
3) Calculate the difference between the resulting image and the original.
4) Apply binary thresholding to obtain a mask with the hairs in the image.
5) Replace the pixels in common between the mask and the original image, with pixels from the latter.
"""

# IMAGE ACQUISITION

# Input image
path = 'Images_GDB/ISIC_0000043_thumbnail_256.jpg'
# Read image
image = cv2.imread(path, cv2.IMREAD_COLOR)
# Image cropping
img = image[30:410, 30:560]

# DULL RAZOR (REMOVE HAIR)

# Gray scale
grayScale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
# Black hat filter
kernel = cv2.getStructuringElement(1, (9, 9))
blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
# Gaussian filter
bhg = cv2.GaussianBlur(blackhat, (3, 3), cv2.BORDER_DEFAULT)
# Binary thresholding (MASK)
ret, mask = cv2.threshold(bhg, 10, 255, cv2.THRESH_BINARY)
# Replace pixels of the mask
dst = cv2.inpaint(img, mask, 6, cv2.INPAINT_TELEA)

# Display images
cv2.imshow("Original image", image)
cv2.imshow("Cropped image", img)
cv2.imshow("Gray Scale image", grayScale)
cv2.imshow("Blackhat", blackhat)
cv2.imshow("Binary mask", mask)
cv2.imshow("Clean image", dst)

cv2.waitKey()
cv2.destroyAllWindows()

TypeError: ignored

**Shade Removal**

In [ ]:
import cv2
import numpy as np


img = cv2.imread(path, -1)

rgb_planes = cv2.split(img)

result_planes = []
result_norm_planes = []
for plane in rgb_planes:
    dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = 255 - cv2.absdiff(plane, bg_img)
    norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
    result_planes.append(diff_img)
    result_norm_planes.append(norm_img)

result = cv2.merge(result_planes)
result_norm = cv2.merge(result_norm_planes)

cv2.imwrite('shadows_out.png', result)
cv2.imwrite('shadows_out_norm.png', result_norm)

**Seuil Adaptatif pour la detection de contours**

In [ ]:
import cv2

def detect_mole_contours(image_path):
    # Charger l'image en niveaux de gris
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Appliquer un seuillage adaptatif pour améliorer les contours
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Réduire le bruit avec une ouverture morphologique
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Trouver les contours dans l'image
    contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Créer une copie de l'image pour dessiner les contours
    img_contours = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # Dessiner les contours sur l'image
    cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)

    # Afficher l'image originale avec les contours détectés
    cv2.imshow("Contours", img_contours)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Exemple d'utilisation
image_path = "Images_GDB/ISIC_0000063_thumbnail_256.jpg"
detect_mole_contours(image_path)

**Histogram Equalizer**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def equalize_histogram(image_path):
    # Charger l'image en niveaux de gris
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Égalisation de l'histogramme
    equalized_img = cv2.equalizeHist(img)

    # Calcul des histogrammes
    hist_img = cv2.calcHist([img], [0], None, [256], [0, 256])
    hist_equalized = cv2.calcHist([equalized_img], [0], None, [256], [0, 256])

    # Afficher l'image originale et l'image égalisée côte à côte
    fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    axs[0, 0].imshow(img, cmap='gray')
    axs[0, 0].set_title("Image originale")
    axs[0, 0].axis('off')
    axs[0, 1].plot(hist_img, color='black')
    axs[0, 1].set_title("Histogramme de l'image originale")
    axs[0, 1].set_xlim([0, 256])
    axs[1, 0].imshow(equalized_img, cmap='gray')
    axs[1, 0].set_title("Image égalisée")
    axs[1, 0].axis('off')
    axs[1, 1].plot(hist_equalized, color='black')
    axs[1, 1].set_title("Histogramme de l'image égalisée")
    axs[1, 1].set_xlim([0, 256])

    plt.tight_layout()
    plt.show()

def equalize_histogram_rgb(image_path):
  image = cv2.imread(image_path)
  img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
  img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
  img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

  cv2.imshow('avant egalisation', image)
  cv2.imshow('apres egalisation', img_output)
  cv2.waitKey(0)



# Exemple d'utilisation
image_path = "Images_GDB/ISIC_0000042_thumbnail_256.jpg"
#equalize_histogram(image_path)
equalize_histogram_rgb(image_path)

error: ignored

In [ ]:
"""
Critère de beauté d'une belle image :

1) Sans poil
2) Avec Beaucoup de contraste
3) Nette
"""


# IMAGE ACQUISITION

# Input image
path = 'Images_GDB/ISIC_0000043_thumbnail_256.jpg'
# Read image
image = cv2.imread(path, cv2.IMREAD_COLOR)
# Image cropping
img = image[30:410, 30:560]

def dullrazor(img):

    # Gray scale
    grayScale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Black hat filter
    kernel = cv2.getStructuringElement(1, (9, 9))
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    # Gaussian filter
    bhg = cv2.GaussianBlur(blackhat, (3, 3), cv2.BORDER_DEFAULT)
    # Binary thresholding (MASK)
    ret, mask = cv2.threshold(bhg, 10, 255, cv2.THRESH_BINARY)
    # Replace pixels of the mask
    dst = cv2.inpaint(img, mask, 6, cv2.INPAINT_TELEA)

    # Display images
    #cv2.imshow("Original image", image)
    #cv2.imshow("Cropped image", img)
    #cv2.imshow("Gray Scale image", grayScale)
    #cv2.imshow("Blackhat", blackhat)
    #cv2.imshow("Binary mask", mask)
    #cv2.imshow("Clean image", dst)
    #cv2.waitKey()
    #cv2.destroyAllWindows()

    return dst

def equalizer(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_hsv[:, :, 0] = cv2.equalizeHist(img_hsv[:, :, 0])
    image_output = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
    return image_output

def contour(img):

    # Gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Appliquer un seuillage adaptatif pour améliorer les contours
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Réduire le bruit avec une ouverture morphologique
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Trouver les contours dans l'image
    contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Créer une copie de l'image pour dessiner les contours
    img_contours = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # Dessiner les contours sur l'image
    cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)

    return img_contours

def save_result(img):
    cv2.imwrite('result.png', img)


def preporcess(img):

    hairremoved = dullrazor(img)
    equalized = equalizer(hairremoved)
    final = contour(equalized)
    without_equalized = contour(hairremoved)

    save_result(final)
    save_result(without_equalized)

    cv2.imshow("Contours", final)
    cv2.imshow("Without Equalizer", without_equalized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

preporcess(img)

**Débruitage**

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# Reading image from folder where it is stored
img = cv2.imread('../photo hair/bruit.jpg')

# denoising of image saving it into dst image
dst = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 15)

# Plotting of source and destination image
plt.subplot(121), plt.imshow(img)
plt.subplot(122), plt.imshow(dst)

plt.show()

**Détection de symétrie**

In [ ]:
import numpy as np
import cv2

img = cv2.imread('../photo hair/as4.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
ret,thresh = cv2.threshold(blur,70,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

max_cnt = max(contours, key=cv2.contourArea)

ellipse = cv2.fitEllipse(max_cnt)
ellipse_pnts = cv2.ellipse2Poly( (int(ellipse[0][0]),int(ellipse[0][1]) ) ,( int(ellipse[1][0]),int(ellipse[1][1]) ),int(ellipse[2]),0,360,1)
comp = cv2.matchShapes(max_cnt,ellipse_pnts,1,0.0)

if comp < 0.099:
	print("symmetric")
else:
    print("asymmetric")